In [1]:
import numpy as np 
import os 
import cv2 
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
import sklearn.metrics as skmetrics
import pandas as pd

from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import confusion_matrix
from scipy.io import loadmat

import itertools
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_openml

#mnist = fetch_openml('mnist_784')

In [2]:
species = os.listdir('scattered_image_lichen_4_1_2_')

In [3]:
def plot_confusion_matrix(cm, classes,string,directory,dt,normalize=True):
    """
    Function which plots confusion matrix

    Input cm = confusion matrix
    Input classes = array with class labels
    Input string = string to give name of the saved image
    Input directory = string to give directory to save the image
    Input normalize (False) = If true function will give accuracy instead of pure number
    Input Title (Confusion matrix) = title of the image


    Output : None
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    accuracy_score = skmetrics.accuracy_score(y_pred, y_test)
    plt.imshow(cm, cmap=plt.cm.Blues)
    plt.title(' accuracy: ' + str(accuracy_score) + '   best rbf_kernel model: '+ str(dt))
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=90)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if(i==j):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            continue

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig(directory +'confusion_matrix'+string+'.jpg')
    
    
def apply_pca_to_train_and_test_images(train_vct, test_vct, perc = .95):
    """
    INPUT:
    train_vct = training vector of shape (N_samples, dimensionality)
    test_vct = testing vector of shape (N_samples, dimensionality)
    perc = percentage of variance that we want ot preserve 
    
    OUTPUT 
    train_transform = new training vector 
    test_transform = new test vector 
    """
    
    scaler = StandardScaler()
    scaler.fit(train_vct)
    train_vct = scaler.transform(train_vct)
    test_vct = scaler.transform(test_vct)
    pca = PCA(perc)
    pca.fit(train_vct)
    train_transform = pca.transform(train_vct)
    test_transform = pca.transform(test_vct)
    
    return train_transform, test_transform
    

In [33]:
def load_descriptor_from_matfile(ft_path,label_list = species):
    feat = loadmat(ft_path)
    feat = feat['scattered_image']
    lab = ft_path.split('/')[1]
    #print(type(lab))
    #print(lab.replace("_", " "))
    lab = label_list.index(lab)
    return feat, lab


def extract_features(labels_list, or_pt):
    data_all = []
    data_mean = []
    label = []
    for i, tp in enumerate(labels_list):
        pth = os.path.join(or_pt, tp)
        mat_files = os.listdir(pth)
        for j,mt in enumerate(mat_files):
            path = os.path.join(pth,mt)
            x,y = load_descriptor_from_matfile(path)
            xall = x.reshape(-1)
            xmean = np.sum(np.sum(x,axis = 2),axis = 1)
            print(xmean.shape)
            xmean = xmean.reshape(-1)
            data_all.append(xall)
            data_mean.append(xmean)
            label.append(y)

    data_all = np.array(data_all)
    data_mean = np.array(data_mean)
    label = np.array(label) 
    return data_all, data_mean, label


In [35]:
c = ['4']


for cc in c:
    print('--------')
    print(cc)
    f = 'scattered_image_lichen_4_' + cc + '_2_'
    labels_list = os.listdir(f)
    title_lich = 'lichen_JLM_4'+ cc + '2_'
    data_all, data_mean, label = extract_features(labels_list,f)
    # PCA REDUCTION 
    X_train, X_test, y_train, y_test = train_test_split(data_mean, label, test_size=0.70, random_state=10, shuffle=True)
    svc  = OneVsRestClassifier(SVC(kernel = 'linear',gamma = 'scale'),n_jobs = -1)
    svc = svc.fit(X_train, y_train)
    linear_score = svc.score(X_test,y_test)
    print(linear_score)
    
    # First grid search to find best parameters
    param_grid = {'C': [ 1e2, 1e3, 5e3, 1e4, 5e4], 'gamma': [ 1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 0.0001, 0.0005, 0.001, 0.005]}
    clf0 = GridSearchCV(SVC(kernel='rbf'), param_grid)
    clf = clf0.fit(X_train, y_train)
    print("Best estimator found by grid search : ", clf.best_estimator_)
    y_pred = clf.predict(X_test)
    print('Accuracy score :', skmetrics.accuracy_score(y_pred, y_test))
    
    plt.figure(figsize = (15,10)) 

    plot_confusion_matrix(skmetrics.confusion_matrix(y_pred, y_test),labels_list,title_lich,'results/',clf.best_estimator_)
    
    

--------
4
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(3

(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)

KeyboardInterrupt: 

In [ ]:
c = ['1','2','3','4','5','6','7','8']


for cc in c:
    print('--------')
    print(cc)
    f = 'scattered_image_lichen_4_' + cc + '_2_'
    labels_list = os.listdir(f)
    title_lich = 'lichen_JLM_4'+ cc + '2_'
    data_all, data_mean, label = extract_features(labels_list,f)
    # PCA REDUCTION 
    X_train, X_test, y_train, y_test = train_test_split(data_mean, label, test_size=0.70, random_state=10, shuffle=True)
    svc  = OneVsRestClassifier(SVC(kernel = 'linear',gamma = 'scale'),n_jobs = -1)
    svc = svc.fit(X_train, y_train)
    linear_score = svc.score(X_test,y_test)
    print(linear_score)
    
    # First grid search to find best parameters
    param_grid = {'C': [  1e3, 5e3, 1e4], 'gamma': [ 1e-5, 0.0001, 0.0005, 0.001],'degree':[2,3,4,5,6]}
    clf0 = GridSearchCV(SVC(kernel='poly'), param_grid)
    clf = clf0.fit(X_train, y_train)
    print("Best estimator found by grid search : ", clf.best_estimator_)
    y_pred = clf.predict(X_test)
    print('Accuracy score :', skmetrics.accuracy_score(y_pred, y_test))
    
    plt.figure(figsize = (15,10)) 

    plot_confusion_matrix(skmetrics.confusion_matrix(y_pred, y_test),labels_list,title_lich,'results/',clf.best_estimator_)
    
    

In [17]:
!pip install kymatio

In [18]:
from kymatio.numpy import Scattering2D



## END-TO-END PROCESS HERE IN PYTHON 

In [20]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import pylab as pl
from sklearn.metrics import confusion_matrix,accuracy_score

DEFINE TRAINING PATH

In [21]:
train_path="../../../patches/train"
class_names=os.listdir(train_path)

In [22]:
class_dir = {}
for i,c in enumerate(class_names):
    class_dir[c] = i

In [47]:
image_paths=[]
image_classes=[]

In [48]:
def img_list(path):
    return (os.path.join(path,f) for f in os.listdir(path))

In [49]:
for training_name in class_names:
    dir_=os.path.join(train_path,training_name)
    class_path=img_list(dir_)
    image_paths+=class_path
    for i in range(len(os.listdir(dir_))):
        image_classes.append(class_dir[training_name])

In [50]:
len(image_classes)

1573

APPEND ALL IMAGE PATH WITH ITS CORRESPONDING LABEL IN A LIST

In [57]:
D=[]
for i in range(len(image_paths)):
    D.append((image_paths[i],image_classes[i]))

In [68]:

D[0]


image_paths, y_train = zip(*D)
#image_paths_test, y_test = zip(*test)

## FEATURE EXTRACTION 




In [91]:
import cv2
def extract_features(D, jj = 4,l = 4 , image_shape = (3,100,100),sp =1 ):
    s = Scattering2D(J=jj, L = l, shape=(100, 100))
    N = len(D)
    sh1 = 1 + l*jj + (l*l*jj*(jj-1))//2
    sh2 = 100//(2**jj)
    sh3 = 100//(2**jj)
    out = []
    lab = []
    des_list = []
    for i,c in enumerate(D):
        lab.append(c[1])
        img = cv2.imread(c[0])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(100,100))
        img = np.array(img)
        img = np.divide(img,255)
        im = np.rollaxis(img, 2, 1)
        img = np.rollaxis(im,1,0)  
        cc = s.scattering(img)
        cc = cc.reshape(cc.shape[0]*cc.shape[1],cc.shape[2],cc.shape[3])
        xmean = np.sum(np.sum(cc,axis = 2),axis = 1)
        if(i%100==0):
            print(xmean.shape)
        xmean = xmean.reshape(-1)        
        out.append(xmean)
        des_list.append((c,xmean))
    out = np.array(out)
    lab = np.array(lab)
    return out,lab,des_list
    

In [92]:
train_data,train_label,des_list = extract_features(dataset)

(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)
(339,)


In [98]:
train_data.shape

(1573, 339)

### Performing K Means clustering on Descriptors

In [118]:
from scipy.cluster.vq import kmeans,vq
from sklearn.preprocessing import StandardScaler

def BOVW(descriptor, k = 400):
    voc,variance=kmeans(descriptor,k,1)
    im_features=np.zeros((len(descriptor),k),"float32")
    c =des_list[0][1].shape[0]
    for i in range(len(descriptor)):
        print(voc.shape)
        print(descriptor[i].shape)
        words,distance=vq(descriptor[i],voc)
        for w in words:
            im_features[i][w]+=1
    stdslr=StandardScaler().fit(im_features)
    im_features=stdslr.transform(im_features)
    return im_features

In [119]:
x_feat = BOVW(train_data)

(400, 339)
(339,)


ValueError: observation and code should have same number of dimensions

In [113]:
X_train, X_test, y_train, y_test = train_test_split(x_feat, train_label, test_size=0.50, random_state=10, shuffle=True)
svc  = OneVsRestClassifier(SVC(kernel = 'poly',gamma = 'scale'),n_jobs = -1)
svc = svc.fit(X_train, y_train)
linear_score = svc.score(X_test,y_test)
print(linear_score)
    

0.49555273189326554


In [121]:
from sklearn.cluster import KMeans

def get_visual_words(descriptors, k):
    """
    Cluster the a sample of the descriptors into visual words.
    @param descriptors: a pandas DataFrame with all the descriptors
    @param k: the number of clusters
    @param n_sampled: the number of descriptors to sample
    @param num_descriptors: total number of descriptors
    @return: an object of type sklearn.cluster.KMea
    """
    kmeans = KMeans(n_clusters=k, n_init=5, n_jobs=-1)
    kmeans = kmeans.fit(np.array(descriptors))
    return kmeans

In [122]:
x_feat = get_visual_words(train_data,200)

/anaconda3/lib/python3.7/site-packages/sklearn/cluster/_kmeans.py:939: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


In [145]:
def kmeans(k, descriptor_list):
    kmeans = KMeans(n_clusters = k, n_init=10)
    kmeans.fit(descriptor_list)
    visual_words = kmeans.cluster_centers_ 
    return visual_words

In [146]:
def find_index(image, center):
    count = 0
    ind = 0
    for i in range(len(center)):
        if(i == 0):
            count = distance.euclidean(image, center[i]) 
           #count = L1_dist(image, center[i])
        else:
            dist = distance.euclidean(image, center[i]) 
            #dist = L1_dist(image, center[i])
            if(dist < count):
                ind = i
                count = dist
    return ind

In [156]:
def image_class(all_bovw, centers):
    dict_feature = []
    for i,img in enumerate(all_bovw):
        print(i)
        #category = []

        histogram = np.zeros(len(centers))
        for each_feature in img:
            ind = find_index(each_feature, centers)
            histogram[ind] += 1

        dict_feature.append(histogram)
    return dict_feature

In [157]:
vw = kmeans(200,train_data)

In [ ]:
from scipy.spatial import distance
ft = image_class(train_data,vw)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27